# Computation
The labels associated with ``DataArray`` and ``Dataset`` objects enables some powerful shortcuts for computation, notably including aggregation and broadcasting by dimension names.

## Basic array math

Arithmetic operations with a single DataArray automatically vectorize (like numpy) over all array values:

In [3]:
%matplotlib inline
import numpy as np
import xarray as xr

In [4]:
arr = xr.DataArray(np.random.randn(2, 3), [('x', ['a', 'b']), ('y', [10, 20, 30])])

In [5]:
arr - 3

<xarray.DataArray (x: 2, y: 3)>
array([[-1.4732171 , -2.50018762, -2.3582823 ],
       [-3.65101496, -4.03530835, -3.6216578 ]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

In [6]:
abs(arr)

<xarray.DataArray (x: 2, y: 3)>
array([[ 1.5267829 ,  0.49981238,  0.6417177 ],
       [ 0.65101496,  1.03530835,  0.6216578 ]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

You can also use any of numpy’s or scipy’s many ufunc functions directly on a DataArray:

In [8]:
np.sin(arr)

<xarray.DataArray (x: 2, y: 3)>
array([[ 0.99903157,  0.47926088,  0.59857232],
       [-0.60599409, -0.86001974, -0.58238361]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

Data arrays also implement many ``numpy.ndarray`` methods:

In [9]:
arr.round(2)

<xarray.DataArray (x: 2, y: 3)>
array([[ 1.53,  0.5 ,  0.64],
       [-0.65, -1.04, -0.62]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

In [10]:
arr.T

<xarray.DataArray (y: 3, x: 2)>
array([[ 1.5267829 , -0.65101496],
       [ 0.49981238, -1.03530835],
       [ 0.6417177 , -0.6216578 ]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

## Missing values
xarray objects borrow the ``isnull()``, ``notnull()``, ``count()``, ``dropna()`` and ``fillna()`` methods for working with missing data from pandas:

In [11]:
x = xr.DataArray([0, 1, np.nan, np.nan, 2], dims=['x'])

In [12]:
x.isnull()

<xarray.DataArray (x: 5)>
array([False, False,  True,  True, False], dtype=bool)
Coordinates:
  * x        (x) int64 0 1 2 3 4

In [13]:
x.notnull()

<xarray.DataArray (x: 5)>
array([ True,  True, False, False,  True], dtype=bool)
Coordinates:
  * x        (x) int64 0 1 2 3 4

In [14]:
x.count()

<xarray.DataArray ()>
array(3)

In [15]:
x.dropna(dim='x')

<xarray.DataArray (x: 3)>
array([ 0.,  1.,  2.])
Coordinates:
  * x        (x) int64 0 1 4

In [16]:
x.fillna(-1)

<xarray.DataArray (x: 5)>
array([ 0.,  1., -1., -1.,  2.])
Coordinates:
  * x        (x) int64 0 1 2 3 4

Like pandas, xarray uses the float value ``np.nan`` (not-a-number) to represent missing values.

# Aggregation
Aggregation methods have been updated to take a *dim* argument instead of *axis*. This allows for very intuitive syntax for aggregation methods that are applied along particular dimension(s):

In [17]:
arr.sum(dim='x')

<xarray.DataArray (y: 3)>
array([ 0.87576794, -0.53549597,  0.0200599 ])
Coordinates:
  * y        (y) int32 10 20 30

In [18]:
arr.std(['x', 'y'])

<xarray.DataArray ()>
array(0.8993685282752704)

In [19]:
arr.min()

<xarray.DataArray ()>
array(-1.0353083542944768)

If you need to figure out the axis number for a dimension yourself (say, for wrapping code designed to work with numpy arrays), you can use the ``get_axis_num()`` method:

In [21]:
arr.get_axis_num('y')

1

These operations automatically skip missing values, like in pandas:

In [22]:
xr.DataArray([1, 2, np.nan, 3]).mean()

<xarray.DataArray ()>
array(2.0)

If desired, you can disable this behavior by invoking the aggregation method with ``skipna=False``.

## Rolling window operations
``DataArray`` objects include a ``rolling()`` method. This method supports rolling window aggregation:

In [23]:
arr = xr.DataArray(np.arange(0, 7.5, 0.5).reshape(3, 5), dims=('x', 'y'))

In [24]:
arr

<xarray.DataArray (x: 3, y: 5)>
array([[ 0. ,  0.5,  1. ,  1.5,  2. ],
       [ 2.5,  3. ,  3.5,  4. ,  4.5],
       [ 5. ,  5.5,  6. ,  6.5,  7. ]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3 4

``rolling()`` is applied along one dimension using the name of the dimension as a key (e.g. ``y``) and the window size as the value (e.g. ``3``). We get back a ``Rolling`` object:

In [25]:
arr.rolling(y=3)

DataArrayRolling [window->3,center->False,dim->y]

The label position and minimum number of periods in the rolling window are controlled by the ``center`` and ``min_periods`` arguments:

In [48]:
arr.rolling(y=3, min_periods=2, center=True)

DataArrayRolling [window->3,min_periods->2,center->True,dim->y]

In [49]:
r = arr.rolling(y=3)

In [50]:
r.mean()

<xarray.DataArray (x: 3, y: 5)>
array([[ nan,  nan,  0.5,  1. ,  1.5],
       [ nan,  nan,  3. ,  3.5,  4. ],
       [ nan,  nan,  5.5,  6. ,  6.5]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3 4

In [47]:
r.reduce(np.std)

<xarray.DataArray (y: 5, x: 3)>
array([[ 0.25      ,  0.25      ,  0.25      ],
       [ 0.40824829,  0.40824829,  0.40824829],
       [ 0.40824829,  0.40824829,  0.40824829],
       [ 0.40824829,  0.40824829,  0.40824829],
       [        nan,         nan,         nan]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3 4

Note that rolling window aggregations are much faster (both asymptotically and because they avoid a loop in Python) when **bottleneck** is installed. Otherwise, we fall back to a slower, pure Python implementation.

Finally, we can manually iterate through ``Rolling`` objects:

In [36]:
for label, arr_window in r:
    print('==============================\n', label, '\n-------------------------\n', arr_window)

 <xarray.DataArray 'y' ()>
array(0, dtype=int64)
Coordinates:
    y        int64 0 
-------------------------
 <xarray.DataArray (x: 3, y: 1)>
array([[ nan],
       [ nan],
       [ nan]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0
 <xarray.DataArray 'y' ()>
array(1, dtype=int64)
Coordinates:
    y        int64 1 
-------------------------
 <xarray.DataArray (x: 3, y: 2)>
array([[ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1
 <xarray.DataArray 'y' ()>
array(2, dtype=int64)
Coordinates:
    y        int64 2 
-------------------------
 <xarray.DataArray (x: 3, y: 3)>
array([[ 0. ,  0.5,  1. ],
       [ 2.5,  3. ,  3.5],
       [ 5. ,  5.5,  6. ]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2
 <xarray.DataArray 'y' ()>
array(3, dtype=int64)
Coordinates:
    y        int64 3 
-------------------------
 <xarray.DataArray (x: 3, y: 3)>
array([[ 0.5,  1. ,  1.5],
  

## Broadcasting by dimension name
``DataArray`` objects are automatically align themselves (“broadcasting” in the numpy parlance) by dimension name instead of axis order. With xarray, you do not need to transpose arrays or insert dimensions of length 1 to get array operations to work, as commonly done in numpy with ``np.reshape()`` or ``np.newaxis``.

This is best illustrated by a few examples. Consider two one-dimensional arrays with different sizes aligned along different dimensions:

In [51]:
a = xr.DataArray([1, 2], [('x', ['a', 'b'])])

In [52]:
a

<xarray.DataArray (x: 2)>
array([1, 2])
Coordinates:
  * x        (x) <U1 'a' 'b'

In [53]:
b = xr.DataArray([-1, -2, -3], [('y', [10, 20, 30])])

In [54]:
b

<xarray.DataArray (y: 3)>
array([-1, -2, -3])
Coordinates:
  * y        (y) int32 10 20 30

With xarray, we can apply binary mathematical operations to these arrays, and their dimensions are expanded automatically:

In [55]:
a * b

<xarray.DataArray (x: 2, y: 3)>
array([[-1, -2, -3],
       [-2, -4, -6]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

Moreover, dimensions are always reordered to the order in which they first appeared:

In [56]:
c = xr.DataArray(np.arange(6).reshape(3, 2), [b['y'], a['x']])

In [57]:
c

<xarray.DataArray (y: 3, x: 2)>
array([[0, 1],
       [2, 3],
       [4, 5]])
Coordinates:
  * y        (y) int32 10 20 30
  * x        (x) <U1 'a' 'b'

In [58]:
a + c

<xarray.DataArray (x: 2, y: 3)>
array([[1, 3, 5],
       [3, 5, 7]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int32 10 20 30

This means, for example, that you always subtract an array from its transpose:

In [59]:
c - c.T

<xarray.DataArray (y: 3, x: 2)>
array([[0, 0],
       [0, 0],
       [0, 0]])
Coordinates:
  * y        (y) int32 10 20 30
  * x        (x) <U1 'a' 'b'